In [17]:
from arcgis.gis import GIS
from azure.storage.queue import QueueService
import json
import logging

import sys
sys.path.append('../src')
sys.path.append('../')

from reach_tools import Reach, ReachFeatureLayer, ReachPointFeatureLayerm
import config

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# create the connection to the GIS to get access
gis = GIS(username=config.arcgis_username, password=config.arcgis_password)
logging.info(f'Connected to GIS at {gis.url}')

# create a layer instance to query and get all existing features
lyr_centroid = ReachFeatureLayer(config.url_reach_centroid, gis)
logging.info(f'Successfully created {lyr_centroid.properties.name} layer.')

# creat a connection to the Azure Queue
queue = QueueService(connection_string=config.azure_queue_conn_str)
logging.info(f'Successfully connected to the {config.azure_queue_update_name} Queue')

In [23]:
msg = queue.get_messages(config.azure_queue_update_name)[0]
msg.content

'{"attributes": {"OBJECTID": 15, "reach_id": "3"}}'

In [24]:
%%time
# we need a connection to arcgis, so start there
gis = GIS(username=config.arcgis_username, password=config.arcgis_password)
logging.info(f'Connected to GIS at {gis.url}.')

# since the input message content is a single feature, extract the reach id from the feature
feature_json = json.loads(msg.content)
reach_id = feature_json['attributes']['reach_id']
logging.info(f'Preparing to update reach id {reach_id}.')

# create a reach object to work with
reach = Reach.get_from_aw(reach_id)
logging.info(f'Retrieved reach id {reach_id} from AW.')

# do the hard work, trace it
reach.snap_putin_and_takeout_and_trace(gis=gis)
logging.info(f'Successfully traced {reach_id}.')

# create layers to be updated
lyr_centroid = ReachFeatureLayer(config.url_reach_centroid, gis)
lyr_line = ReachFeatureLayer(config.url_reach_line, gis)
lyr_points = ReachPointFeatureLayer(config.url_reach_points, gis)

# update ArcGIS Online
reach.publish_updates(lyr_line, lyr_centroid, lyr_points)
logging.info(f'Successfully updated {reach_id} on ArcGIS Online.')

CPU times: user 865 ms, sys: 57.8 ms, total: 923 ms
Wall time: 42.1 s
